In [1]:
import random
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

class PSOOptimize(object):
    def __init__(self, no_cities, data):
        self.iter_max = 50# No of Iterations
        self.num = 500  # No of particles
        self.no_cities = no_cities  # Num of cities
        self.coordinate = data # city location coordinates
        # Calculate distance matrix
        self.dis_mat = self.comp_dis_mat(no_cities, self.coordinate)  # calculate dist matrix between cities
        # Initialize all particles
        self.particals = self.greedy_initialization(self.dis_mat,count_t=self.num,no_cities =no_cities)
        self.lenths,self.mean_length,self.median_length = self.comp_routes(self.particals)
        # Get optimal solution
        init_l = min(self.lenths)
        init_index = self.lenths.index(init_l)
        init_route = self.particals[init_index]
        # Drae the initial route
        init_show = self.coordinate[init_route]
        # Record curr optimal solution for each individual
        self.local_best = self.particals
        self.local_best_len = self.lenths
        # Record curr global optimal solution
        self.global_best = init_route
        self.global_best_len = init_l
        # Output solution
        self.best_l = self.global_best_len
        self.best_route = self.global_best
        # Store output of each iteration
        self.loop_x = [0]
        self.loop_y = [init_l]
        #self.mean_length = None
        #self.median_length = None
    def greedy_initialization(self, dis_mat, count_t, no_cities):
        start_idx = 0
        outcome = []
        for j in range(count_t):
            rest = [i for i in range(0, no_cities)]
            
            if start_idx >= no_cities:
                start_idx = np.random.randint(0, no_cities)
                outcome.append(outcome[start_idx].copy())
                continue
            curr = start_idx
            rest.remove(curr)
            # Find the nearest neighbor route
            outcome_one = [curr]
            while len(rest) != 0:
                dist_min = math.inf
                dist_select = -1
                for k in rest:
                    if dis_mat[curr][k] < dist_min:
                        dist_min = dis_mat[curr][k]
                        dist_select = k

                curr = dist_select
                outcome_one.append(dist_select)
                rest.remove(dist_select)
            outcome.append(outcome_one)
            start_idx += 1
        return outcome

    # random initialization

    def random_init(self, count_t, no_cities):

        dist = [k for k in range(no_cities)]
        outcome = []
        for j in range(count_t):
            random.shuffle(dist)
            outcome.append(dist.copy())
        return outcome

    # Calculate distance between cities
    def comp_dis_mat(self, no_cities, coordinate):
        dis_mat = np.zeros((no_cities, no_cities))
        for i in range(no_cities):
            for j in range(no_cities):
                if i == j:
                    dis_mat[i][j] = np.inf
                    continue
                c = coordinate[i]
                d = coordinate[j]
                dist = np.sqrt(sum([(x[0] - x[1]) ** 2 for x in zip(c, d)]))
                dis_mat[i][j] = dist
        return dis_mat

    # Calculate the length of route
    def comp_routelen(self, route, dis_mat):

        c = route[0]
        d = route[-1]
        outcome = dis_mat[c][d]

        for j in range(len(route) - 1):
            c = route[j]
            d = route[j + 1]
            outcome += dis_mat[c][d]

        return outcome

    # Calculate group length
    def comp_routes(self, routes):
        outcome = []
        mean=[]
        median=[]
        for i in routes:
            length = self.comp_routelen(i, self.dis_mat)
            outcome.append(length)
            mean.append(np.mean(outcome))
            median.append(np.median(outcome))
        self.mean_length = np.mean(outcome)
        self.median_length = np.median(outcome)
        return outcome,mean,median

    # Assess the curr group
    def eval_particals(self):
        min_lenth = min(self.lenths)
        min_index = self.lenths.index(min_lenth)
        cur_route = self.particals[min_index]

        # Update curr global optimum
        if min_lenth < self.global_best_len:
            self.global_best_len = min_lenth
            self.global_best = cur_route
        # Update curr individual best
        for j, k in enumerate(self.lenths):
            if k < self.local_best_len[j]:
                self.local_best_len[j] = k
                self.local_best[j] = self.particals[j]
        self.mean_length = np.mean(self.lenths)
        self.median_length = np.median(self.lenths)

    # cross over
    def cross(self, cur, best):
        one = cur.copy()
        l = [t for t in range(self.no_cities)]
        t = np.random.choice(l,2)
        x = min(t)
        y = max(t)
        cross_part = best[x:y]
        dist = []
        for t in one:
            if t in cross_part:
                continue
            dist.append(t)
        one = dist + cross_part
        l1 = self.comp_routelen(one, self.dis_mat)
        one2 = cross_part + dist
        l2 = self.comp_routelen(one2, self.dis_mat)
        if l1<l2:
            return one, l1
        else:
            return one, l2


    # particle mutation

    def mutate(self, one):
        one = one.copy()
        l = [t for t in range(self.no_cities)]
        t = np.random.choice(l, 2)
        x, y = min(t), max(t)
        one[x], one[y] = one[y], one[x]
        l2 = self.comp_routelen(one,self.dis_mat)
        return one, l2

    # iterative operation
    def pso(self):
        best = []
        outcomes_dict = {
            "generation": [],
            "best_fitness": [],
            "mean_fitness": [],
            "median_fitness": [],

        }

        for cnt in range(1, self.iter_max+1):
            for i, one in enumerate(self.particals):
                dist_l = self.lenths[i]
                # Crossover with the curr individual local optimal solution
                new_one, new_l = self.cross(one, self.local_best[i])

                if new_l < self.best_l:
                    self.best_l = dist_l
                    self.best_route = one

                if new_l < dist_l or np.random.rand()<0.1:
                    one = new_one
                    dist_l = new_l

                # Crossover with the curr global optimal solution
                new_one, new_l = self.cross(one, self.global_best)

                if new_l < self.best_l:
                    self.best_l = dist_l
                    self.best_route = one

                if new_l < dist_l or np.random.rand()<0.1:
                    one = new_one
                    dist_l = new_l
                # Mutation
                one, dist_l = self.mutate(one)

                if new_l < self.best_l:
                    self.best_l = dist_l
                    self.best_route = one

                if new_l < dist_l or np.random.rand()<0.1:
                    one = new_one
                    dist_l = new_l

                # update the particle
                self.particals[i] = one
                self.lenths[i] = dist_l
            # Evaluate particle swarms, update individual local optimum and individual curr global optimum
            self.eval_particals()
            # update output solution
            if self.global_best_len < self.best_l:
                self.best_l = self.global_best_len
                self.best_route = self.global_best
            #print(cnt, self.best_l)
            self.loop_x.append(cnt)
            self.loop_y.append(self.best_l)
            #print(i,self.best_l," mean ",self.mean_length,"median ",self.median_length)
            outcomes_dict["generation"].append(cnt)
            outcomes_dict["best_fitness"].append(self.best_l)
            outcomes_dict["mean_fitness"].append(self.mean_length)
            outcomes_dict["median_fitness"].append(self.median_length)
        return self.best_l, self.best_route,outcomes_dict

    def run(self):
        best_length, best_route,outcomes = self.pso()
        # final route
        return self.coordinate[best_route], best_length,outcomes


# Read the data
def read_tsp(route):
    rows = open(route, 'r').readlines()
    assert 'NODE_COORD_SECTION\n' in rows
    idx = rows.index('NODE_COORD_SECTION\n')
    tsp = rows[idx + 1:-1]
    dist = []
    for i in tsp:
        i = i.strip().split(' ')
        if i[0] == 'EOF':
            continue
        distline = []
        for k in i:
            if k == '':
                continue
            else:
                distline.append(float(k))
        if distline == []:
            continue
        dist.append(distline)
    tsp = dist
    return tsp


dataset = read_tsp('/content/st70.tsp')

dataset = np.array(dataset)
dataset = dataset[:, 1:]
show_data = np.vstack([dataset, dataset[0]])
all_outcomes = []
import pandas as pd
for i in range(0,30):
 random.seed(i)
 model = PSOOptimize(no_cities=dataset.shape[0], data=dataset.copy())
 Best_route, Best,outcomes = model.run()
 print(pd.DataFrame(outcomes))
 all_outcomes.append(pd.DataFrame(outcomes))

final_outcomes = pd.concat(all_outcomes, axis=1)
print(final_outcomes)

Best_route = np.vstack([Best_route, Best_route[0]])
plt.plot(Best_route[:, 0], Best_route[:, 1], marker='o', markerfacecolor='red')
plt.title('st70:PSO outcome')
plt.show()
 



KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
final_results.to_csv('file_pso_50iter_new_500pop.csv', index=False)

In [ ]:
final= final_results.drop('generation', axis=1).reset_index()


In [ ]:
import numpy as np

def parseData(data, firstcolumn, noRuns):
    col = firstcolumn
    
    allstats = (data.shape[1]-1)/noRuns   # how many stats were collected. Omit the first column (Generations)
    cols = np.arange(col, noRuns*allstats+1, allstats, dtype=int)
    #print(cols)
    subdata = data.iloc[:,cols]
    # print(subdata)
    noGens = data.shape[0]
    pdata = np.zeros((noGens, 4))
    for i in range(noGens):
        pdata[i,0] = i+1
        pdata[i,1] = np.mean(subdata.iloc[i,:].mean()) # mean(subdata[i,])
        pdata[i,2] = 1.96*np.std(subdata.iloc[i,:])/np.sqrt(noRuns) # 1.96*sd(rowMeans(subdata))/sqrt(noRuns) # compute the length of error bar. 
        pdata[i,3] = np.min(subdata.iloc[i,:]) # mean(subdata[i,])
    return pdata


In [ ]:
  data = parseData(final, 1, 30)
  mean_second_col = np.mean(data[:,1])
  min_second_col = np.min(data[:,1])

  print("mean ",mean_second_col)
  print("min ",min_second_col)

mean  735.693522039723
min  725.994365107932
